In [1]:
!nvidia-smi

Fri May  7 11:20:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    29W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import sys
print(sys.version_info)

sys.version_info(major=3, minor=7, micro=10, releaselevel='final', serial=0)


In [3]:
import time
START = time.time()

In [4]:
!pip install transformers==4.5

In [5]:
!cp drive/MyDrive/zindi_nlp/MalawiNews/*.csv .

In [6]:
! pip install sentencepiece

In [7]:
import torch
import os
import sys
from torch import nn
import transformers
from transformers import AutoTokenizer, AutoModel
from torch.nn import functional as F
from sklearn.metrics import accuracy_score
from tqdm.autonotebook import tqdm
from torch.cuda.amp import GradScaler, autocast
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import gc

In [8]:
import random
import numpy as np 
import torch
from tqdm.notebook import tqdm
from collections import Counter
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


SEED = 42

def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

      
    
seed_everything(SEED)

In [9]:
print(torch.__version__)

1.8.1+cu101


# Utils

In [10]:
def train_clf(model, dataloader, optimizer,  loss_fn, cfg, scheduler=None):
  scaler = GradScaler()
  accumulation = cfg.accumulation

  t = tqdm(dataloader, disable= not cfg.verbose)
  total_loss = 0
  model.train()
  
  for i, batch in enumerate(t):
    x, y = batch

    x = x.to(cfg.device, dtype=torch.long)
    y = y.to(cfg.device, dtype=torch.long)
    with autocast(cfg.use_apex):
      outputs = model(x)

    loss = loss_fn(outputs, y)

    total_loss += loss.item()


    if cfg.use_apex:
        loss = loss/accumulation
        scaler.scale(loss).backward()
    else:
        loss = loss/accumulation
        loss.backward()

    if (i+1)%accumulation == 0 or i-1 == len(t):
        if cfg.use_apex:
            scaler.step(optimizer)

            # Updates the scale for next iteration.
            scaler.update()
            optimizer.zero_grad()
        else:                
            optimizer.step()
            optimizer.zero_grad()
            

    if scheduler is not None:
      scheduler.step()  
    t.set_description("Loss : {0} ".format(total_loss/(i+1)) )
    t.refresh()       

  

In [11]:
def eval_clf(model, dataloader, loss_fn, criterion, cfg):
  total_loss = 0.
  t=tqdm(dataloader, disable= not cfg.verbose_val)
  y_true = []
  y_preds = []
  model.eval()
  device = cfg.device
  with torch.no_grad():
      for i, batch in enumerate(t):
          
          inputs,  labels = batch
          
          inputs = inputs.to(device, dtype=torch.long)
          labels = labels.to(device, dtype=torch.long)
          with autocast(cfg.use_apex):
            outputs = model(inputs)
          
          loss = loss_fn(outputs, labels ) 
          total_loss += loss.detach().cpu().numpy()
          
          t.set_description("Loss : {0}".format(total_loss/(i+1)))
          t.refresh()
      
          y_true.append(labels.detach().cpu().numpy())
          y_preds.append( outputs.cpu().detach().numpy())


  y_preds = np.concatenate(y_preds)
  y_true = np.concatenate(y_true)
  score = criterion(y_true, y_preds)
  return y_preds, y_true, score, total_loss/(i+1)

In [12]:
def inference_clf(model, dataloader, cfg):
  total_loss = 0.
  t=tqdm(dataloader, disable= not cfg.verbose_val)
  
  y_preds = []
  model.eval()
  device = cfg.device
  with torch.no_grad():
      for i, batch in enumerate(t):
          
          inputs = batch[0]
          
          inputs = inputs.to(device, dtype=torch.long)
           
          outputs = model(inputs)
          y_preds.append( torch.softmax(outputs, axis=-1).cpu().detach().numpy())
  y_preds = np.concatenate(y_preds)
  return y_preds 

In [13]:
def accuracy(y_true, y_preds, one_hot_label=False):
  p = np.argmax(y_preds, axis=1)
  if one_hot_label:
    y_true = np.argmax(y_true, axis=1)
  return accuracy_score(y_true, p)

# Dataset

In [14]:
def encode_texts(texts, tokenizer, max_lengths):
  encode_text = []
  for text in tqdm(texts):
    encode_text.append(tokenizer.encode(text, max_length=max_lengths, padding="max_length", truncation=True, return_attention_mask=False, return_token_type_ids=False))

  return np.stack(encode_text)

class TextDataset(torch.utils.data.Dataset):
    """
        Dataloader test which slices n seconds based on tmin and tmax
    
    """
    def __init__(self, samples, labels=None, tokenizer=None, max_length=130):
        self.labels = None
        self.samples=samples
        self.tokenizer = tokenizer
        if labels is not None:
          self.labels = labels
        self.max_length = max_length
    
    
    def __getitem__(self, index):
        l = []
        
        text = self.samples[index] if self.tokenizer is None else tokenizer.encode(self.samples[index],  max_length=self.max_length, padding="max_length", truncation=True, return_attention_mask=False, return_token_type_ids=False)
        l.append(torch.as_tensor(text).long())

        if self.labels is not None:
            l.append( torch.as_tensor(self.labels[index]))
        #l.append(torch.tensor(index))
        return tuple(l)
    
    def __len__(self):
        return len(self.samples)

class TextDatasetSequence(torch.utils.data.Dataset):
    """
        Dataloader test which slices n seconds based on tmin and tmax
    
    """
    def __init__(self, samples, labels=None, max_length=130, padding=0):
        self.labels = None
        self.samples=samples
        if labels is not None:
          self.labels = labels
        self.max_length = max_length
    
    
    def __getitem__(self, index):
        l = []
        
        text = self.samples[index] 
        length = len(text)

        if self.max_length - length > 0:
          text = np.array(text.tolist() + [self.padding] * (self.max_length - length))

        l.append(torch.as_tensor(text).long())
        l.append(torch.as_tensor(length))
        if self.labels is not None:
            l.append( torch.as_tensor(self.labels[index]))
        #l.append(torch.tensor(index))
        return tuple(l)
    
    def __len__(self):
        return len(self.samples)

# Model

In [15]:
class AutoClassifier(nn.Module):
  def __init__(self, backbone, num_class, padding_idx, hidden_size=768):
    super(AutoClassifier, self).__init__()
    self.padding_idx = padding_idx
    self.backbone = backbone
    self.num_class = num_class
    self.cls = nn.Sequential(nn.Dropout(0.25), nn.Linear(hidden_size, 512), nn.ReLU(), nn.Dropout(0.25), nn.Linear(512, self.num_class))

  def forward(self, input):
    attention_mask  = (~(input == self.padding_idx)).long()
    emb = self.backbone(input, attention_mask=attention_mask)
    emb = emb[0][:,0] # CLS
    return self.cls(emb)

class AutoClassifierM(nn.Module):
  def __init__(self, backbone, num_class, padding_idx, hidden_size=768):
    super(AutoClassifierM, self).__init__()
    self.padding_idx = padding_idx
    self.backbone = backbone
    self.num_class = num_class
    self.cls = nn.Sequential(nn.Dropout(0.25), nn.Linear(hidden_size, 512), nn.ReLU(), nn.Dropout(0.25), nn.Linear(512, self.num_class))

  def forward(self, input):
    attention_mask  = (~(input == self.padding_idx)).long()
    hidden_states = self.backbone(input, attention_mask=attention_mask).last_hidden_state
    eos_mask = input_ids.eq(self.config.eos_token_id)

    if len(torch.unique(eos_mask.sum(1))) > 1:
        raise ValueError("All examples must have the same number of <eos> tokens.")
    sentence_representation = hidden_states[eos_mask, :].view(hidden_states.size(0), -1, hidden_states.size(-1))[:, -1, :]
    return self.cls(sentence_representation)

In [16]:
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn.model_selection import StratifiedKFold

# Training

In [17]:
# 
class2id = {"POLITICS":0, "SOCIAL":1, "RELIGION":2, "LAW/ORDER":3, "SOCIAL ISSUES":4, "HEALTH":5, "ECONOMY":6, "FARMING":7, "SPORTS":8, "EDUCATION":9,
            "RELATIONSHIPS":10, "WILDLIFE/ENVIRONMENT":11, "OPINION/ESSAY":12, "LOCALCHIEFS":13, "CULTURE": 14, "WITCHCRAFT":15, "MUSIC":16, "TRANSPORT":17,
            "FLOODING":18, "ARTS AND CRAFTS":19}
id2class = {x:y for y,x in class2id.items()}  

In [18]:
print(class2id)
print(id2class)

{'POLITICS': 0, 'SOCIAL': 1, 'RELIGION': 2, 'LAW/ORDER': 3, 'SOCIAL ISSUES': 4, 'HEALTH': 5, 'ECONOMY': 6, 'FARMING': 7, 'SPORTS': 8, 'EDUCATION': 9, 'RELATIONSHIPS': 10, 'WILDLIFE/ENVIRONMENT': 11, 'OPINION/ESSAY': 12, 'LOCALCHIEFS': 13, 'CULTURE': 14, 'WITCHCRAFT': 15, 'MUSIC': 16, 'TRANSPORT': 17, 'FLOODING': 18, 'ARTS AND CRAFTS': 19}
{0: 'POLITICS', 1: 'SOCIAL', 2: 'RELIGION', 3: 'LAW/ORDER', 4: 'SOCIAL ISSUES', 5: 'HEALTH', 6: 'ECONOMY', 7: 'FARMING', 8: 'SPORTS', 9: 'EDUCATION', 10: 'RELATIONSHIPS', 11: 'WILDLIFE/ENVIRONMENT', 12: 'OPINION/ESSAY', 13: 'LOCALCHIEFS', 14: 'CULTURE', 15: 'WITCHCRAFT', 16: 'MUSIC', 17: 'TRANSPORT', 18: 'FLOODING', 19: 'ARTS AND CRAFTS'}


In [19]:
def encode_texts_left(texts, tokenizer, max_lengths):
  encode_text = []
  for text in tqdm(texts):
    encoded = tokenizer.encode(text, max_length=max_lengths, padding="max_length", truncation=False, return_attention_mask=False, return_token_type_ids=False)
    length = len(encoded)
    if length > max_lengths:
      to_remove = length - ( max_lengths - 1)
      encoded = [tokenizer.cls_token_id] + encoded[to_remove:]
      if len(encoded) != max_lengths:
        print(len(encoded))
        raise("error in tokenizer left")
    encode_text.append(encoded)

  return np.stack(encode_text)

In [20]:
import json
class Config():
  def __init__(self, name, kfold=5, batch_size=16, accumulation=1, loss_fn=nn.CrossEntropyLoss):
      self.kfold = kfold
      self.device = "cuda" if torch.cuda.is_available() else "cpu"
      self.use_apex = True
      self.batch_size = batch_size
      self.accumulation = accumulation
      self.name = name
      self.ckpt ="XLM-Roberta-large-200-40folds" #  "xlm-roberta" #
      self.save_name = "./" + self.ckpt
      self.save_name_csv = "drive/MyDrive/zindi_nlp/MalawiNews/XLM-r-large/ckpt-80000/" + self.ckpt
      self.num_class = len(class2id)
      self.max_length= 200
      self.epochs= 8 #15
      self.early_stopping =5
      self.warmup = 0.01
      self.verbose = True
      self.verbose_val = False
      # loss function
      self.loss_fn = loss_fn
      self.lr = [2e-5]*self.kfold # 1e-5
      self.wd=3e-5
      self.stop_epoch=50

cfg = Config(name="drive/MyDrive/zindi_nlp/MalawiNews/pretrain/xlm-r-large/checkpoint-80000", batch_size =4, kfold=40, accumulation=8) #
if not os.path.exists(cfg.save_name_csv):
  os.makedirs(cfg.save_name_csv)
#cfg_str = json.dumps(cfg)
if not os.path.exists(cfg.save_name):
  os.makedirs(cfg.save_name)

In [21]:
train = pd.read_csv("Train.csv")
test = pd.read_csv("Test.csv")
sub = pd.read_csv("SampleSubmission.csv")

In [22]:
train["label"] = train.Label.apply(lambda x: class2id[x])

In [23]:
tokenizer = AutoTokenizer.from_pretrained(cfg.name, use_fast=False)


In [24]:
tokenizer

PreTrainedTokenizer(name_or_path='drive/MyDrive/zindi_nlp/MalawiNews/pretrain/xlm-r-large/checkpoint-80000', vocab_size=250002, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})

In [25]:
# Encode test data
test_texts = encode_texts(test.Text, tokenizer, cfg.max_length)
test_dataset = TextDataset(test_texts)
test_dataloader = DataLoader(test_dataset, batch_size=cfg.batch_size, shuffle=False, num_workers=4)

In [26]:
# Encode test data 
test_texts_left = encode_texts_left(test.Text, tokenizer, cfg.max_length)
test_dataset_left = TextDataset(test_texts_left)
test_dataloader_left = DataLoader(test_dataset_left, batch_size=cfg.batch_size, shuffle=False, num_workers=4)

In [27]:
all_texts = encode_texts(train.Text, tokenizer, cfg.max_length)
all_labels = train.label.values

In [28]:
max_ = train.Label.value_counts().max()
weights_name = max_/train.Label.value_counts()
weights = torch.as_tensor([weights_name[id2class[x]] for x in range(len(id2class)) ]).half()

#loss_fn=nn.CrossEntropyLoss()
loss_fn=nn.CrossEntropyLoss(weight=weights).to(cfg.device)

In [29]:
BERTWEET_MODEL=True
ELECTRA_MODEL=False
XLMROBERTA_MODEL = False
mBart_MODEL = False
EVAL_ONLY=False

In [30]:
if BERTWEET_MODEL:
  skf = StratifiedKFold(n_splits=cfg.kfold, random_state=42)
  test_preds = []
  test_preds_left = []
  oof_preds = []
  oof_targets = []
  dataloaders = []
  fold2use = {i:[] for i in range(10)}
  results =[]
  for fold, (train_idx, val_idx) in enumerate(skf.split( np.zeros(len(train)), train.label.values)):
    print(f"#### FOLD : {fold} ####")
    counter = 0
    best_score = 0
    # get train/val
    train_dataset = TextDataset(all_texts[train_idx], labels=all_labels[train_idx], max_length=cfg.max_length)
    val_dataset = TextDataset(all_texts[val_idx], labels=all_labels[val_idx], max_length=cfg.max_length)
    
    train_dataloader = DataLoader(train_dataset, batch_size=cfg.batch_size, shuffle=True, num_workers=4)
    val_dataloader = DataLoader(val_dataset, batch_size=cfg.batch_size*2, shuffle=False, num_workers=1)
    
    # optimizer and model and scheduler
    total_steps = cfg.epochs * len(train_dataloader)
    num_warmup_steps = cfg.warmup * total_steps
    num_training_steps = total_steps
    backbone = AutoModel.from_pretrained(cfg.name)
    model = AutoClassifier(backbone, num_class=cfg.num_class, padding_idx=tokenizer.pad_token_id, hidden_size=1024).to(cfg.device)


    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": cfg.wd,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]

    optimizer = AdamW(optimizer_grouped_parameters, lr=cfg.lr[fold])#, weight_decay=cfg.wd)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps , num_training_steps=num_training_steps)
    if not EVAL_ONLY:
      for e in range(cfg.epochs):
        train_clf(model, train_dataloader, optimizer,  loss_fn, cfg, scheduler=scheduler)
        y_preds ,y_true, val_score, val_loss = eval_clf(model, val_dataloader, loss_fn, accuracy, cfg)

        if val_score > best_score:
          print(f"Improvement from {best_score} to {val_score}, saving model ...")
          best_score = val_score
          torch.save(model.state_dict(), cfg.save_name + "/" +  f"{cfg.ckpt}-fold{fold}.pth")
          counter = 0
        else:
          print("not an improvement :", val_score)
          counter +=1

        if counter > cfg.early_stopping:
          print("========= Early stopping ========")
          break
        if cfg.stop_epoch is not None:
          if e >= cfg.stop_epoch:
            print("========= Early stopping ========")
            break
    model.load_state_dict(torch.load(cfg.save_name + "/" + f"{cfg.ckpt}-fold{fold}.pth"))
    y_preds ,y_true, val_score, val_loss = eval_clf(model, val_dataloader, loss_fn, accuracy, cfg)
    oof_targets.append(y_true)
    oof_preds.append(y_preds)
    preds = inference_clf(model, test_dataloader, cfg)
    preds2 = inference_clf(model, test_dataloader_left, cfg)
    test_preds.append(preds)
    test_preds_left.append((preds+preds2)/2.0)
    dataloaders.append(val_dataloader)
    results.append(val_score)
    os.remove(cfg.save_name + "/" +  f"{cfg.ckpt}-fold{fold}.pth")
    del preds, preds2
    gc.collect()
  test_preds=np.stack(test_preds)
  test_preds_left=np.stack(test_preds_left)

  oof_preds = np.concatenate(oof_preds)
  oof_targets = np.concatenate(oof_targets)
  avg = accuracy(oof_targets, oof_preds)
  print("AVG ACC : ", avg)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 7 members, which is less than n_splits=40.
  % (min_groups, self.n_splits)), UserWarning)


#### FOLD : 0 ####


Some weights of XLMRobertaModel were not initialized from the model checkpoint at drive/MyDrive/zindi_nlp/MalawiNews/pretrain/xlm-r-large/checkpoint-80000 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)



Improvement from 0 to 0.25, saving model ...


...


not an improvement : 0.5714285714285714



Improvement from 0.5714285714285714 to 0.6285714285714286, saving model ...



Improvement from 0.6285714285714286 to 0.6571428571428571, saving model ...



not an improvement : 0.6571428571428571



Improvement from 0.6571428571428571 to 0.6857142857142857, saving model ...
AVG ACC :  0.7263231197771588


In [31]:
print("Training Time : ", print(time.time()-START))
START = time.time()

24908.966825723648
Training Time :  None


In [32]:
with open(cfg.save_name_csv + "/" + f"{cfg.ckpt}-results-{avg}.txt" , "w") as f:
  # f.write(cfg_str)
  # f.write("\n\n")
  for r in results:
    f.write(f"{round(r,4)} \n")
  f.write(str(avg))

In [33]:
name_prob = [f"label_{x}" for x in range(cfg.num_class)]


In [34]:
sub["ID"] = test["ID"]

In [35]:
if BERTWEET_MODEL:
  #sub["ID"] = sub.ID.apply(lambda x : x.zfill(7))
  sub["Label"] = np.argmax(test_preds.mean(0), axis=1) 
  sub["Label"] = sub.Label.apply(lambda x: id2class[x])
  sub.to_csv(cfg.save_name_csv + f"submission-{round(avg,4)}.csv", index=False)
  sub.Label.value_counts()
  sub[name_prob] = test_preds.mean(0) 
  sub.to_csv(cfg.save_name_csv + f"submission-{round(avg,4)}-proba.csv", index=False)

In [36]:
sub.Label.value_counts()

POLITICS                107
LAW/ORDER                72
RELIGION                 66
HEALTH                   62
SOCIAL                   55
SOCIAL ISSUES            53
ECONOMY                  46
FARMING                  32
SPORTS                   22
WILDLIFE/ENVIRONMENT     22
RELATIONSHIPS            21
EDUCATION                19
LOCALCHIEFS              15
MUSIC                    12
CULTURE                   7
TRANSPORT                 6
WITCHCRAFT                3
Name: Label, dtype: int64

# submission with right and left truncation

In [37]:
sub = pd.read_csv("SampleSubmission.csv")
sub["ID"] = test["ID"]

In [38]:
if BERTWEET_MODEL:
  #sub["ID"] = sub.ID.apply(lambda x : x.zfill(7))
  sub["Label"] = np.argmax(test_preds_left.mean(0), axis=1) 
  sub["Label"] = sub.Label.apply(lambda x: id2class[x])
  sub.to_csv(cfg.save_name_csv + f"submission-{round(avg,4)}-left.csv", index=False)
  sub.Label.value_counts()
  sub[name_prob] = test_preds_left.mean(0) 
  sub.to_csv(cfg.save_name_csv + f"submission-{round(avg,4)}-proba-left.csv", index=False)

In [39]:
sub.Label.value_counts()

POLITICS                109
LAW/ORDER                81
RELIGION                 69
HEALTH                   62
SOCIAL                   51
SOCIAL ISSUES            48
ECONOMY                  44
FARMING                  36
SPORTS                   24
RELATIONSHIPS            22
WILDLIFE/ENVIRONMENT     21
EDUCATION                20
LOCALCHIEFS              13
MUSIC                     8
CULTURE                   7
TRANSPORT                 5
Name: Label, dtype: int64